# Aprendizaje No Supervisado con K-Means
## Análisis de Clustering en Datos Industriales IoT

Este notebook demuestra técnicas de **aprendizaje no supervisado** aplicadas a datos reales de sensores industriales.
Aprenderemos a:

1. **Preparar y normalizar datos** de sensores
2. **Determinar el número óptimo de clusters** usando el método del codo y coeficiente de silueta
3. **Visualizar clusters** en 2D y 3D mediante PCA
4. **Interpretar resultados** y realizar análisis estadístico
5. **Guardar y usar modelos** para predicciones en tiempo real

### 📊 Dataset
**Fuente:** [Mendeley Data - Sensor Signals](https://data.mendeley.com/datasets/7rjn2xj7bc/1)  
**Publicado:** 30 Mayo 2020 | Versión 1

**Descripción:**  
Señales de sensores de una aplicación real de IoT industrial. Los datos provienen de sensores de la planta de concentración de mineral de hierro "Fakoor Sanat". El molino de bolas es un equipo crítico y costoso que requiere monitoreo continuo. Los sensores recopilan temperaturas y las envían a la nube Fakoor para prevención de fallas.


## 1. Preparación de Datos y Determinación del Número Óptimo de Clusters

En esta sección:
- Cargaremos y prepararemos los datos de sensores
- Normalizaremos los datos usando `StandardScaler` (importante para K-Means)
- Aplicaremos el **Método del Codo** para visualizar la inercia
- Calcularemos el **Coeficiente de Silueta** para evaluar la calidad del clustering
- Determinaremos el número óptimo de clusters (k)


In [ ]:
# Montar Google Drive (descomentar si usas Google Colab)
# from google.colab import drive
# drive.mount('/content/drive')

## 2. Análisis Estadístico de Clusters

Ahora analizaremos las características de cada cluster para entender qué representan en términos de temperatura de los sensores. Esto nos ayudará a:

- Identificar patrones operacionales
- Detectar estados anómalos
- Tomar decisiones de mantenimiento predictivo


In [ ]:
# ============================================
# 📦 IMPORTACIÓN DE BIBLIOTECAS
# ============================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Suprimir advertencias para visualización limpia
import warnings
warnings.filterwarnings('ignore')

# Configuración de gráficos
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

In [ ]:
# https://data.mendeley.com/datasets/7rjn2xj7bc/1
# Mendeley Data: Sensor Signals. Published: 30 May 2020| Version 1
# Description:
# The data set contains sensor signals from a real application in industrial IoT.
# Our sample data contains data sensors from “Fakoor Sanat Iron Ore Concentrate Plant” that is IoT enabled.
# Ball mill is a piece of important and expensive equipment in the Iron Ore Concentrate industry.
# Sensors collect temperature and send them to the Fakoor cloud. Events gathered from the ball mill are used for failure prevention.

# --- Cargar los datos ---
columnas = ['Sensor1', 'Sensor2', 'Sensor3', 'Sensor4', 'Sensor5', 'Sensor6', 'Sensor7', 'Sensor8']
df = pd.read_csv("ballmill_temp.csv", header=None)
df.columns = columnas

# --- Opcional: Usar muestra menor para acelerar ---
df = df.sample(n=20000, random_state=42)

# # Verificar si hay valores 0 en cualquier columna
# mask = (df == 0).any(axis=1)
# # Mantener solo las filas sin valores 0
# df = df[~mask]

In [ ]:
df.describe().round(2)

In [ ]:
# --- Escalar los datos ---
scaler = StandardScaler()
datos_normalizados = scaler.fit_transform(df)

In [ ]:
pd.DataFrame(datos_normalizados, columns=columnas).describe().round(2)

In [ ]:
# --- Método del codo e índice de silueta ---
inertias = []
silhouette_scores = []
k_values = range(2, 11)

for k in k_values:
    # print('k= ', k)
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    etiquetas = kmeans.fit_predict(datos_normalizados)

    inertias.append(kmeans.inertia_)
    sil_score = silhouette_score(datos_normalizados, etiquetas, sample_size=1000, random_state=42)
    silhouette_scores.append(sil_score)

# --- Gráfico del Método del Codo ---
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(k_values, inertias, marker='o')
plt.title('Método del Codo')
plt.xlabel('Número de Clusters (k)')
plt.ylabel('Inercia')
plt.grid(True)

# --- Gráfico del Coeficiente de Silueta ---
plt.subplot(1, 2, 2)
plt.plot(k_values, silhouette_scores, marker='o', color='green')
plt.title('Coeficiente de Silueta')
plt.xlabel('Número de Clusters (k)')
plt.ylabel('Silhouette Score')
plt.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# ================================
# 🎯 Visualización 3D de clusters
# ================================

# --- Definir k final (con base en los gráficos anteriores) ---
k_final = 4  # 👈 Ajusta este valor según el mejor k encontrado

colores = [
    '#1f77b4',  # Cluster 0 - Azul
    '#2ca02c',  # Cluster 1 - Verde
    '#ff00ff',  # Cluster 2 - Violeta
    '#ff7f0e',  # Cluster 3 - Naranja
    # '#d62728'   # Cluster 3 - Rojo
]

# --- Recalcular KMeans con k_final ---
kmeans_final = KMeans(n_clusters=k_final, random_state=42, n_init=10)
etiquetas_finales = kmeans_final.fit_predict(datos_normalizados)

# Crear lista de colores por punto
colores_por_punto = [colores[cluster_id] for cluster_id in etiquetas_finales]

# --- Reducción de dimensionalidad con PCA para 3D ---
pca = PCA(n_components=3)
datos_pca = pca.fit_transform(datos_normalizados)

# --- Gráfico 3D de los clusters ---
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')

scatter = ax.scatter(
    datos_pca[:, 0], datos_pca[:, 1], datos_pca[:, 2],
    c=colores_por_punto, s=10, alpha=0.7
)

ax.set_title(f'Clusters visualizados en 3D (k={k_final})')
ax.set_xlabel('PCA 1')
ax.set_ylabel('PCA 2')
ax.set_zlabel('PCA 3')
#plt.colorbar(scatter, ax=ax, label='Cluster')
plt.show()

In [ ]:
# ================================
# 📊 Visualización 2D de clusters
# ================================

# --- Reducir a 2 dimensiones con PCA ---
pca = PCA(n_components=2)
datos_pca = pca.fit_transform(datos_normalizados)

# --- Graficar clusters en 2D ---
plt.figure(figsize=(6, 5))
scatter = plt.scatter(
    datos_pca[:, 0], datos_pca[:, 1],
    c=colores_por_punto, s=10, alpha=0.7
)

plt.title(f'Clusters visualizados en 2D con PCA (k={k_final})')
plt.xlabel('Componente Principal 1')
plt.ylabel('Componente Principal 2')
#plt.colorbar(scatter, label='Cluster')
plt.grid(True)
plt.tight_layout()
plt.show()

## 3. Visualización Detallada de Estadísticas

Vamos a crear visualizaciones más detalladas para entender mejor la distribución de temperaturas en cada cluster.


In [ ]:
# ============================================
# 📈 ANÁLISIS ESTADÍSTICO POR CLUSTER
# ============================================

# Agregar las etiquetas de cluster al DataFrame original
df['Cluster'] = etiquetas_finales

# Contar muestras por cluster
conteo_clusters = df['Cluster'].value_counts().sort_index()
print("🔢 Cantidad de muestras por cluster:")
print(conteo_clusters)
print(f"\n📊 Porcentaje por cluster:")
print((conteo_clusters / len(df) * 100).round(2))

In [ ]:
# Calcular estadísticas por cluster
print("\n📊 Medias de temperatura por cluster:")
medias_por_cluster = df.groupby('Cluster')[columnas].mean()
medias_por_cluster.round(2)

In [ ]:
print("\n📉 Desviaciones estándar por cluster:")
desvios_por_cluster = df.groupby('Cluster')[columnas].std()
desvios_por_cluster.round(2)

In [ ]:
# ============================================
# 📊 VISUALIZACIÓN DE MEDIAS POR CLUSTER
# ============================================
plt.figure(figsize=(12, 6))

for cluster_id in medias_por_cluster.index:
    plt.plot(columnas,
             medias_por_cluster.loc[cluster_id],
             marker='o',
             linewidth=2,
             markersize=8,
             label=f'Cluster {cluster_id}',
             color=colores[cluster_id] if cluster_id < len(colores) else None)

plt.title('🌡️ Temperatura Media de Sensores por Cluster', fontsize=14, fontweight='bold')
plt.xlabel('Sensores', fontsize=12)
# plt.ylabel('Temperatura Media (°C)', fontsze=12)
plt.grid(True, alpha=0.3)
plt.legend(fontsize=11)
plt.tight_layout()
plt.show()

In [ ]:
# ============================================
# 💡 INTERPRETACIÓN DE CLUSTERS
# ============================================
print("\n💡 INTERPRETACIÓN DE CLUSTERS:")
print("="*60)
for cluster_id in range(len(conteo_clusters)):
    temp_media = medias_por_cluster.loc[cluster_id].mean()
    num_muestras = conteo_clusters[cluster_id]
    porcentaje = (num_muestras / len(df) * 100)

    print(f"\nCluster {cluster_id}:")
    print(f"  - Temperatura media: {temp_media:.2f}°C")
    print(f"  - Muestras: {num_muestras} ({porcentaje:.2f}%)")

    if temp_media < 10:
        print(f"  - Estado: ⚠️ ALERTA - Sensores inactivos o error")
    elif temp_media < 40:
        print(f"  - Estado: ✅ Operación normal - Temperatura baja")
    elif temp_media < 55:
        print(f"  - Estado: ⚡ Operación normal - Temperatura media")
    else:
        print(f"  - Estado: 🔥 ATENCIÓN - Temperatura elevada")

# ============================================
# 💾 GUARDAR MODELOS ENTRENADOS
# ============================================
print("\n💾 Guardando modelos...")

# Guardar scaler (normalización)
joblib.dump(scaler, 'scaler.pkl')
print("  ✅ Scaler guardado como 'scaler.pkl'")

# Guardar PCA (opcional, para visualización)
joblib.dump(pca, 'pca.pkl')
print("  ✅ PCA guardado como 'pca.pkl'")

# Guardar modelo K-Means
joblib.dump(kmeans_final, 'kmeans.pkl')
print("  ✅ K-Means guardado como 'kmeans.pkl'")

print("\n✅ Todos los modelos guardados con éxito!")
print("📝 Estos archivos se pueden usar para hacer predicciones en nuevos datos")


## 4. Predicción en Tiempo Real

Ahora simularemos cómo usar los modelos guardados para predecir el cluster de nuevas lecturas de sensores. Esto es útil para:
- Sistemas de monitoreo en tiempo real
- Detección de anomalías
- Alertas automáticas


In [ ]:
# ============================================
# 📊 ANÁLISIS DETALLADO CON BOXPLOT
# ============================================

# Preparar datos en formato largo para visualización
df_largo = df.melt(id_vars=['Cluster'],
                   value_vars=columnas,
                   var_name='Sensor',
                   value_name='Temperatura')

# Crear boxplot por cluster
plt.figure(figsize=(14, 6))
sns.boxplot(x='Cluster', y='Temperatura', data=df_largo,
            palette=colores, showfliers=False)
plt.title('📊 Distribución de Temperaturas por Cluster',
         fontsize=14, fontweight='bold')
plt.xlabel('Cluster', fontsize=12)
plt.ylabel('Temperatura (°C)', fontsize=12)
plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

# ============================================
# 🎻 VIOLINPLOT PARA VER DISTRIBUCIONES
# ============================================
plt.figure(figsize=(14, 6))
sns.violinplot(x='Cluster', y='Temperatura', data=df_largo,
               palette=colores[:len(conteo_clusters)])
plt.title('🎻 Distribución Detallada de Temperaturas por Cluster',
         fontsize=14, fontweight='bold')
plt.xlabel('Cluster', fontsize=12)
plt.ylabel('Temperatura (°C)', fontsize=12)
plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

# ============================================
# 🔥 HEATMAP DE TEMPERATURAS MEDIAS
# ============================================
plt.figure(figsize=(10, 6))
sns.heatmap(medias_por_cluster, annot=True, fmt='.1f', cmap='YlOrRd',
           cbar_kws={'label': 'Temperatura (°C)'})
plt.title('🔥 Mapa de Calor: Temperatura Media por Cluster y Sensor',
         fontsize=14, fontweight='bold')
plt.xlabel('Sensor', fontsize=12)
plt.ylabel('Cluster', fontsize=12)
plt.tight_layout()
plt.show()

print("\n✅ Visualizaciones detalladas completadas")


## 5. Conclusiones y Recomendaciones

### 🎓 Conceptos Aprendidos

En este notebook hemos explorado el **aprendizaje no supervisado** aplicado a datos industriales reales. Los conceptos clave incluyen:

#### 1. **K-Means Clustering**
- Algoritmo particional que agrupa datos similares
- Requiere especificar el número de clusters (k)
- Sensible a la escala de los datos (requiere normalización)
- Iterativo: asigna puntos → calcula centroides → repite

#### 2. **Determinación del Número Óptimo de Clusters**
- **Método del Codo**: Busca el punto donde la inercia deja de disminuir significativamente
- **Coeficiente de Silueta**: Mide qué tan bien cada punto está asignado a su cluster (rango: -1 a 1)
- Valores de silueta > 0.5 indican buen clustering

#### 3. **PCA (Análisis de Componentes Principales)**
- Técnica de reducción de dimensionalidad
- Encuentra direcciones de máxima varianza
- Permite visualizar datos de alta dimensión en 2D/3D
- Preserva la mayor información posible

### 🏭 Aplicaciones Industriales

Los resultados de este análisis tienen aplicaciones prácticas importantes:

#### **Mantenimiento Predictivo**
- Detectar patrones que preceden a fallos
- Identificar estados operacionales anómalos
- Programar mantenimiento basado en condición

#### **Optimización de Procesos**
- Identificar estados operacionales óptimos
- Reducir consumo energético
- Maximizar eficiencia del equipo

#### **Monitoreo en Tiempo Real**
- Sistema de alertas automáticas
- Detección temprana de anomalías
- Respuesta rápida a condiciones críticas

#### **Optimización de Sensores**
- Identificar sensores redundantes
- Reducir costos de instrumentación
- Mantener cobertura de monitoreo adecuada

### 📊 Resultados Clave de este Análisis

1. **Clusters Identificados**: Se encontraron 4 grupos principales de estados operacionales
2. **Interpretación**:
   - **Cluster 0**: Operación normal con temperaturas medias (~45°C)
   - **Cluster 1**: Operación con temperaturas elevadas (~61°C) - requiere atención
   - **Cluster 2**: Sensores inactivos (0°C) - posible falla
   - **Cluster 3**: Operación con temperaturas bajas (~30°C)

3. **Correlación de Sensores**: Análisis jerárquico muestra qué sensores tienen comportamiento similar

### 🚀 Próximos Pasos Recomendados

#### **A Corto Plazo**
1. Implementar sistema de alertas automáticas en producción
2. Validar clusters con expertos del dominio
3. Establecer umbrales de temperatura para cada cluster

#### **A Mediano Plazo**
1. Análisis temporal: estudiar evolución de clusters en el tiempo
2. Implementar modelos adicionales:
   - **DBSCAN**: Para detectar clusters de forma arbitraria
   - **Isolation Forest**: Para detección de anomalías
   - **One-Class SVM**: Para identificar outliers
3. Integrar con sistemas SCADA/MES existentes

#### **A Largo Plazo**
1. Desarrollo de gemelo digital del proceso
2. Optimización automática de parámetros operacionales
3. Predicción de vida útil del equipo
4. Integración con mantenimiento preventivo programado

### 📚 Recursos Adicionales

**Librerías Utilizadas:**
- `scikit-learn`: Machine Learning
- `pandas`: Manipulación de datos
- `matplotlib/seaborn`: Visualización
- `scipy`: Análisis científico
- `joblib`: Persistencia de modelos

**Para Profundizar:**
- [Documentación de scikit-learn](https://scikit-learn.org/)
- [Clustering en Machine Learning](https://scikit-learn.org/stable/modules/clustering.html)
- [PCA Explicado](https://scikit-learn.org/stable/modules/decomposition.html#pca)

### 💡 Consejos Finales

1. **Siempre normaliza** los datos antes de aplicar K-Means
2. **No confíes ciegamente** en métricas automáticas - valida con expertos
3. **Visualiza** los resultados desde múltiples perspectivas
4. **Documenta** las decisiones y parámetros utilizados
5. **Itera** - el análisis de datos es un proceso iterativo

---

## ✅ Notebook Completado

Has aprendido a aplicar técnicas de aprendizaje no supervisado a datos industriales reales. Estos métodos son fundamentales para:
- Descubrir patrones ocultos en los datos
- Tomar decisiones basadas en datos
- Optimizar procesos industriales
- Implementar mantenimiento predictivo

**¡Felicidades por completar este análisis!** 🎉
